In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

# Load the trained model
model = load_model('C:/Users/panch/AppData/Local/Programs/Python/Python311/Lib/site-packages/model.h5')

# Define a function to preprocess the input image
def preprocess_img(img):
    img = cv2.resize(img, (150, 150))  # Resize the frame to match the model's input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to predict tourist spot
def predict_tourist_spot(img):
    img_array = preprocess_img(img)
    predictions = model.predict(img_array)
    decoded_predictions = custom_decode_predictions(predictions)
    return decoded_predictions

# Custom decoding function
def custom_decode_predictions(predictions):
    # You should implement this function based on your model's classes
    # This is just an example assuming your model predicts probabilities for each class
    class_labels = ['Bull Temple', 'Commercial Street', 'Freedom Park', 'HAL', 'ISRO', 'Jawaharlal Nehru Planetarium', 'Kempegowda bus Terminal, Majestic', 'Kempegowda International Airport', 'M G Road', 'National Gallery of Modern Art', 'St. Marys Basilica', 'UB City Mall', 'Visvesvaraya Industrial and Technological Museum']
    decoded_predictions = [(label, prob) for label, prob in zip(class_labels, predictions[0])]
    decoded_predictions.sort(key=lambda x: x[1], reverse=True)  # Sort by probability
    return decoded_predictions

# Open a camera
cap = cv2.VideoCapture(0)

# Set the confidence threshold for displaying predictions
confidence_threshold = 0.8  # You can adjust this threshold

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('Camera Feed', frame)

    # Make predictions on the current frame
    predictions = predict_tourist_spot(frame)

    # Display the top prediction on the frame if confidence is above the threshold
    top_prediction = predictions[0]
    if top_prediction[1] > confidence_threshold:
        cv2.putText(frame, f'{top_prediction[0]}: {top_prediction[1]:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Print and display predictions
    print(predictions)
    cv2.imshow('Predictions', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()